In [88]:
%matplotlib widget

import sympy as sp
import sympy.physics.mechanics as me
import sympy.plotting as splt
from typing import List
from sympy import sin, cos, pi, sqrt, acos, simplify, atan
import math
me.init_vprinting()


def homogeneous(rotation: sp.Matrix = sp.eye(3), translation: sp.Matrix = sp.zeros(3, 1)) -> sp.Matrix:
    return rotation.row_join(translation).col_join(sp.Matrix([[0, 0, 0, 1]]))

def dh(rotation, twist, displacement, offset):
    rotation_mat = sp.Matrix([
        [cos(rotation), -sin(rotation)*cos(twist),  sin(rotation)*sin(twist)],
        [sin(rotation), cos(rotation)*cos(twist),   -cos(rotation)*sin(twist)],
        [0,             sin(twist),                 cos(twist)],
    ])
    translation = sp.Matrix([
        [offset*cos(rotation)],
        [offset*sin(rotation)],
        [displacement],
    ])
    return rotation_mat, translation

def rotation(homogeneous: sp.Matrix):
    return homogeneous[:3, :3]

def translation(homogeneous: sp.Matrix):
    return homogeneous[:3, 3:]

def chained_transform(transforms: List[sp.Matrix]):
    transforms_chained = [homogeneous()]
    for transform in transforms:
        transforms_chained.append(transforms_chained[-1] * transform)
    return transforms_chained

def z_vecs(transforms: List[sp.Matrix]):
    transforms_chained = chained_transform(transforms)
    z_unit_vecs = []
    for transform in transforms_chained:
        z_unit_vecs.append(rotation(transform) * sp.Matrix([0, 0, 1]))
    return z_unit_vecs

def jacobian(transforms: List[sp.Matrix], joint_types: List[sp.Matrix], base_z: sp.Matrix = sp.Matrix([0, 0, 1])):
    transforms_chained = chained_transform(transforms)
    z_unit_vecs = z_vecs(transforms)
    
    assert len(transforms_chained) == len(z_unit_vecs)

    jacobian = sp.zeros(6, len(transforms))
    for i, (transform, joint_type) in enumerate(zip(transforms, joint_types)):
        if joint_type == 'revolute':
            jacobian[:3, i] = z_unit_vecs[i].cross(translation(transforms_chained[-1]) - translation(transforms_chained[i]))
            jacobian[3:, i] = z_unit_vecs[i]
        elif joint_type == 'prismatic':
            jacobian[:3, i] = z_unit_vecs[i]
            jacobian[3:, i] = sp.Matrix([[0], [0], [0]])

        # angular velocity

    return jacobian

def skew(v: sp.Matrix):
    return sp.Matrix([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0],
    ])

In [89]:
t = sp.symbols('t')
g = sp.symbols('g')
# joint variables
# theta_1, d_1 = sp.symbols('\\theta_1, d_1')
theta_1, d_2 = me.dynamicsymbols('\\theta_1, d_2')
q = sp.Matrix([theta_1, d_2])
q_dot = q.diff(t)
# physical properties
l_c1, m_1, m_2, I_1, I_2 = sp.symbols('l_c1, m_1, m_2, I_1, I_2')
r_c = [sp.Matrix([0, 0, 0, 1]), sp.Matrix([0, 0, l_c1, 1]), sp.Matrix([0, 0, 0, 1])] # CoM positions in the joint frame
m = [None, m_1, m_2]
I = [None, I_1, I_2]

joint_1 = homogeneous(*dh(pi/2 + theta_1, pi/2, 0, 0))
joint_2 = homogeneous(*dh(0, 0, d_2, 0))
all_joints = [joint_1, joint_2]
joint_types = ['revolute', 'prismatic']

In [90]:
display(joint_1)
display(joint_2)
display(joint_1*sp.Matrix([0, 0, 0, 1]))
display(joint_1*joint_2)

⎡-sin(\theta₁)  0  cos(\theta₁)  0⎤
⎢                                 ⎥
⎢cos(\theta₁)   0  sin(\theta₁)  0⎥
⎢                                 ⎥
⎢      0        1       0        0⎥
⎢                                 ⎥
⎣      0        0       0        1⎦

⎡1  0  0  0 ⎤
⎢           ⎥
⎢0  1  0  0 ⎥
⎢           ⎥
⎢0  0  1  d₂⎥
⎢           ⎥
⎣0  0  0  1 ⎦

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣1⎦

⎡-sin(\theta₁)  0  cos(\theta₁)  d₂⋅cos(\theta₁)⎤
⎢                                               ⎥
⎢cos(\theta₁)   0  sin(\theta₁)  d₂⋅sin(\theta₁)⎥
⎢                                               ⎥
⎢      0        1       0               0       ⎥
⎢                                               ⎥
⎣      0        0       0               1       ⎦

In [91]:
# Compute Jacobian
J = jacobian(all_joints, joint_types)

w = [sp.Matrix([0, 0, 0])]*(len(all_joints)+1) # joint linear velocities
v = [sp.Matrix([0, 0, 0])]*(len(all_joints)+1) # joint angular velocities
v_c = [sp.Matrix([0, 0, 0])]*(len(all_joints)+1) # joint CoM linear velocities
T = [sp.Matrix([0])]*(len(all_joints)+1) # joint kinetic energy
V = [sp.Matrix([0])]*(len(all_joints)+1) # joint potential energy

chained_transforms = chained_transform(all_joints)
# chained_translations = chained_translation(all_joints)
# z = z_vecs(all_joints) # joint origins
z = sp.Matrix([0, 0, 1]) # base z vector

for i, joint, joint_type in zip(range(1, len(all_joints) + 1), all_joints, joint_types):
    # Compute angular velocity
    theta_dot = q_dot[i-1] if joint_type == 'revolute' else 0
    w[i] = rotation(joint).T * (w[i-1] + z*theta_dot)

    # Compute linear velocity
    d_dot = q_dot[i-1] if joint_type == 'prismatic' else 0
    r_i = (joint*sp.Matrix([0, 0, 0, 1]))[:3, :]
    v[i] = rotation(joint).T * (v[i-1] + z*d_dot) + w[i].cross(r_i)

    # Compute CoM linear velocity
    v_c[i] = v[i] + w[i].cross(r_c[i][:3, :])

    # Compute kinetic energy
    T[i] = 0.5*m[i]*v_c[i].T*v_c[i] + 0.5*w[i].T*I[i]*w[i]

    # Compute potential energy
    p_ci = (chained_transforms[i]*r_c[i])[:3, :]
    V[i] = -m[i]*sp.Matrix([0, -g, 0]).T*p_ci

display(J)
display(w[1])
display(w[2])
display(v[1])
display(v[2])
display(v_c[1])
display(v_c[2])
display(T[1])
display(T[2])
display(V[1])
display(V[2])

⎡-d₂⋅sin(\theta₁)  cos(\theta₁)⎤
⎢                              ⎥
⎢d₂⋅cos(\theta₁)   sin(\theta₁)⎥
⎢                              ⎥
⎢       0               0      ⎥
⎢                              ⎥
⎢       0               0      ⎥
⎢                              ⎥
⎢       0               0      ⎥
⎢                              ⎥
⎣       1               0      ⎦

⎡   0   ⎤
⎢       ⎥
⎢\thėta₁⎥
⎢       ⎥
⎣   0   ⎦

⎡   0   ⎤
⎢       ⎥
⎢\thėta₁⎥
⎢       ⎥
⎣   0   ⎦

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

⎡d₂⋅\thėta₁⎤
⎢          ⎥
⎢    0     ⎥
⎢          ⎥
⎣    d₂̇    ⎦

...

⎡d₂⋅\thėta₁⎤
⎢          ⎥
⎢    0     ⎥
⎢          ⎥
⎣    d₂̇    ⎦

⎡              2           2           2⎤
⎣0.5⋅I₁⋅\thėta₁  + 0.5⋅l_c1 ⋅m₁⋅\thėta₁ ⎦

⎡              2            2        2            2⎤
⎣0.5⋅I₂⋅\thėta₁  + 0.5⋅m₂⋅d₂ ⋅\thėta₁  + 0.5⋅m₂⋅d₂̇ ⎦

[g⋅l_c1⋅m₁⋅sin(\theta₁)]

[g⋅m₂⋅d₂⋅sin(\theta₁)]

In [92]:
# Construct robot dynamics
T = T[1]+T[2]
V = V[1]+V[2]
L = T - V

f_x, f_y, f_z, g_x, g_y, g_z = sp.symbols('f_x, f_y, f_z, g_x, g_y, g_z')
F_ext = sp.Matrix([f_x, f_y, f_z, g_x, g_y, g_z])
# to = L.diff(q_dot).diff(t) - L.diff(q)

temp = L.diff(q_dot).diff(t) - L.diff(q)
temp = sp.Matrix([temp[0][0][0][0], temp[1][0][0][0]])
temp - J.T*F_ext

⎡                                                                             
⎢1.0⋅I₁⋅\thëta₁ + 1.0⋅I₂⋅\thëta₁ + fₓ⋅d₂⋅sin(\theta₁) - f_y⋅d₂⋅cos(\theta₁) 
⎢                                                                             
⎢                                                                             
⎣                                                       -fₓ⋅cos(\theta₁) - f_y

                                                              2               
+ g⋅l_c1⋅m₁⋅cos(\theta₁) + g⋅m₂⋅d₂⋅cos(\theta₁) - g_z + 1.0⋅l_c1 ⋅m₁⋅\thëta₁ 
                                                                              
                                                 2                            
⋅sin(\theta₁) + g⋅m₂⋅sin(\theta₁) - m₂⋅d₂⋅\thėta₁  + 1.0⋅m₂⋅d₂̈              

        2                               ⎤
+ 1.0⋅m₂⋅d₂ ⋅\thëta₁ + 2.0⋅m₂⋅d₂⋅\thėta₁⋅d₂̇⎥
                                        ⎥
                                        ⎥
                                  